## 使用_ _slots_ _

正常情况下，当我们定义了一个class，创建了一个class的实例后，我们可以给该实例绑定任何属性和方法，这就是动态语言的灵活性

In [1]:
class Student(object):
    pass

In [2]:
s = Student()
s.name = 'Micheal'

In [3]:
print(s.name)

Micheal


In [5]:
def set_age(self, age):
    self.age = age

In [6]:
from types import MethodType
s.set_age = MethodType(set_age, s)
s.set_age(25)
print(s.age)

25


但是，给一个实例绑定的方法，对另一个实例是不起作用的  
为了给所有实例都绑定方法，可以给class绑定方法

In [7]:
s2 = Student()
s2.set_age(25)

AttributeError: 'Student' object has no attribute 'set_age'

In [8]:
def set_score(self, score):
    self.score = score
Student.set_score = set_score

In [9]:
s.set_score(100)
print(s.score)

100


In [10]:
s2.set_score(99)
print(s2.score)

99


### 使用_ _slots_ _

但是，如果我们想要限制实例的属性怎么办？比如，只允许对Student实例添加name和age属性  
为了达到限制的目的，Python允许在定义class的时候，定义一个特殊的_ _slots_ _变量，来限制该class实例能添加的属性  
使用_ _slots_ _要注意，_ _slots_ _定义的属性仅对当前类实例起作用，对继承的子类是不起作用的

In [11]:
class Student(object):
    __slots__ = ('name', 'age')

In [12]:
s = Student()
s.name = 'Micheal'
s.age = 25
s.score = 99

AttributeError: 'Student' object has no attribute 'score'

In [13]:
class GraduateStudent(Student):
    pass
g = GraduateStudent()
g.score = 99

## 使用@property

在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改

In [14]:
class Student(object):
    def get_score(self):
        return self._score
    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer')
        if value< 0 or value > 100:
            raise ValueError('score must between 0~100')
        self._score = value

In [15]:
s = Student()
s.set_score(60)
s.get_score()

60

In [16]:
s.set_score(10000)

ValueError: score must between 0~100

有没有既能检查参数，又可以用类似属性这样简单的方式来访问类的变量呢  
Python内置的@property装饰器就是负责把一个方法变成属性调用的

In [17]:
class Student(object):
    @property
    def score(self):
        return self._score
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer')
        if value < 0 or value > 100:
            raise ValueError('score must between 0~100')
        self._score = value

In [18]:
s = Student()
s.score = 60
print(s.score)
s.score = 9999

60


ValueError: score must between 0~100

@property的实现比较复杂，我们先考察如何使用。把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身又创建了另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控的属性操作  
还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性

In [19]:
class Student(object):
    @property
    def birth(self):
        return self._birth
    @birth.setter
    def birth(self, value):
        self._birth = value
    @property
    def age(self):
        return 2017 - self._birth

In [23]:
# _*_ coding: utf-8 _*_
class Screen(object):
    @property
    def width(self):
        return self._width
    @width.setter
    def width(self, value):
        self._width = value
    @property
    def height(self):
        return self._height
    @height.setter
    def height(self, value):
        self._height = value
    @property
    def resolution(self):
        return self._width * self._height

In [24]:
#测试
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过')
else:
    print('测试失败')

resolution = 786432
测试通过


## 多重继承  
通过多重继承，一个子类就可以同时获得多个父类的所有功能

In [25]:
class Animal(object):
    pass

#大类
class Mammal(Animal):
    pass
class Bird(Animal):
    pass

#各种动物
class Dog(Mammal):
    pass
class Bat(Mammal):
    pass
class Parrot(Bird):
    pass
class Ostrich(Bird):
    pass

In [26]:
class Runnable(object):
    def run(self):
        print('Running...')
class Flyable(object):
    def fly(self):
        print('Flying...')

In [27]:
class Dog(Mammal, Runnable):
    pass
class Bat(Mammal, Flyable):
    pass

### MixIn  
为了更好地看出继承关系，我们把Runnable和Flyable改为RunnableMixIn和FlyableMixIn。类似的，你还可以定义出肉食动物CarnivorousMixIn和植食动物HerbivoresMixIn，让某个动物同时拥有好几个MixIn  
MixIn的目的就是给一个类增加多个功能，这样，在设计类的时候，我们优先考虑通过多重继承来组合多个MixIn的功能，而不是设计多层次的复杂的继承关系

In [28]:
class Dog(Mammal, RunableMixIn, CarnivorousMixIn):
    pass

NameError: name 'RunableMixIn' is not defined

Python自带的很多库也使用了MixIn。举个例子，Python自带了TCPServer和UDPServer这两类网络服务，而要同时服务多个用户就必须使用多进程或多线程模型，这两种模型由ForkingMixIn和ThreadingMixIn提供。通过组合，我们就可以创造出合适的服务来

In [30]:
class MyTcpServer(TCPServer, ForkingMixIn):
    pass
class MyUDPServer(UDPServer, ThreadingMixIn):
    pass
class MyTCPServer(TCPServer, CoroutineMixIn):
    pass

NameError: name 'TCPServer' is not defined

## 定制类

### _ _str_ _

In [1]:
class Student(object):
    def __init__(self, name):
        self.name = name
print(Student('Michael'))

In [8]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name, %s)' % self.name
s = Student('Micheal')
print(s)
s

Student object (name, Micheal)


这是因为直接显示变量调用的不是_ _str_ _()，而是_ _repr_ _()，两者的区别是_ _str_ _()返回用户看到的字符串，而_ _repr_ _()返回程序开发者看到的字符串，也就是说，_ _repr_ _()是为调试服务的

In [10]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' % self.name
    __repr__ = __str__
s = Student('Micheal')
print(s)
s

Student object (name=Micheal)


Student object (name=Micheal)

### _ _iter_ _

如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个_ _iter_ _()方法，该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的 _ _next_ _()方法拿到循环的下一个值，直到遇到StopIteration错误时退出

In [12]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1
    def __iter__(self):
        return self
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        if self.a > 100000:
            raise StopIteration()
        return self.a

In [14]:
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


### _ _getitem_ _

In [15]:
Fib()[5]

TypeError: 'Fib' object does not support indexing

要表现得像list那样按照下标取出元素，需要实现_ _getitem_ _()方法：

In [16]:
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a

In [18]:
f = Fib()
for x in range(10):
    print(f[x])
print(f[100])

1
1
2
3
5
8
13
21
34
55
573147844013817084101


In [19]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int):
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice):#原因是__getitem__()传入的参数可能是一个int，也可能是一个切片对象slice，所以要做判断
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

In [20]:
f = Fib()
print(f[0:5])
print(f[:10])

[1, 1, 2, 3, 5]
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


### _ _getattr_ _

In [27]:
class Student(object):
    def __init__(self):
        self.name = 'Michael'
    def __getattr__(self, attr):
        if attr == 'score':
            return 99

In [28]:
s = Student()
print(s.name)
print(s.score)
print(s.abc)

Michael
99
None


注意，只有在没有找到属性的情况下，才调用_ _getattr_ _，已有的属性，比如name，不会在_ _getattr_ _中查找  
此外，注意到任意调用如s.abc都会返回None，这是因为我们定义的_ _getattr_ _默认返回就是None。要让class只响应特定的几个属性，我们就要按照约定，抛出AttributeError的错误

In [31]:
class Student(object):
    def __getattr__(self, attr):
        if attr == 'age':
            return lambda: 25
        raise AttributeError('\'Student\' object has no attribute \'%s\'' %attr)

In [32]:
s =Student()
print(s.age)
print(s.abc)

<function Student.__getattr__.<locals>.<lambda> at 0x0000000006369B70>


AttributeError: 'Student' object has no attribute 'abc'

In [35]:
class Chain(object):
    def __init__(self, path=''):
        self._path =path
    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))
    def __str__(self):
        return self._path
    __repr__ = __str__

In [36]:
Chain().status.user.timeline.list

/status/user/timeline/list

### _ _call_ _

当我们调用实例方法时，我们用instance.method()来调用。能不能直接在实例本身上调用呢？在Python中，答案是肯定的  
任何类，只需要定义一个_ _call_ _()方法，就可以直接对实例进行调用

In [37]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __call__(self):
        print('My name is %s.' % self.name)

In [44]:
s = Student('Micheal')
s()

My name is Micheal.


那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个Callable对象

In [47]:
print(callable(Student('Micheal')))
print(callable(max))
print(callable([1, 2, 3]))
print(callable(None))

True
True
False
False


## 使用枚举类

枚举类型定义一个class类型，然后，每个常量都是class的一个唯一实例。Python提供了Enum类来实现这个功能  
这样我们就获得了Month类型的枚举类，可以直接使用Month.Jan来引用一个常量，或者枚举它的所有成员

In [48]:
from enum import Enum
Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

In [50]:
for name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Aug => Month.Aug , 7
Sep => Month.Sep , 8
Oct => Month.Oct , 9
Nov => Month.Nov , 10
Dec => Month.Dec , 11


value属性则是自动赋给成员的int常量，默认从1开始计数  
如果需要更精确地控制枚举类型，可以从Enum派生出自定义类  
@unique装饰器可以帮助我们检查保证没有重复值

In [51]:
from enum import Enum, unique
@unique
class Weekday(Enum):
    Sun = 0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

In [55]:
day1 = Weekday.Mon
print(day1)
print(Weekday['Tue'])
print(Weekday.Tue.value)

Weekday.Mon
Weekday.Tue
2


In [58]:
print(Weekday(0))
for name, member in Weekday.__members__.items():
    print(name, '=>', member)

Weekday.Sun
Sun => Weekday.Sun
Mon => Weekday.Mon
Tue => Weekday.Tue
Wed => Weekday.Wed
Thu => Weekday.Thu
Fri => Weekday.Fri
Sat => Weekday.Sat


In [61]:
# _*_ coding: utf-8 _*_
from enum import Enum, unique
class Gender(Enum):
    Male = 0
    Female = 1
class Student(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender

In [62]:
#测试
bart = Student('Bart', Gender.Male)
if bart.gender == Gender.Male:
    print('测试通过')
else:
    print('测试失败')

测试通过


### 使用元类

In [67]:
from hello import Hello
h = Hello()
h.hello()
print(type(Hello))
print(type(h))

Hello, world.
<class 'type'>
<class 'hello.Hello'>


我们说class的定义是运行时动态创建的，而创建class的方法就是使用type()函数  
type()函数既可以返回一个对象的类型，又可以创建出新的类型，比如，我们可以通过type()函数创建出Hello类，而无需通过class Hello(object)...的定义

要创建一个class对象，type()函数依次传入3个参数：
1,class的名称；
2,继承的父类集合，注意Python支持多重继承，如果只有一个父类，别忘了tuple的单元素写法；
3,class的方法名称与函数绑定，这里我们把函数fn绑定到方法名hello上

In [69]:
def fn(self, name='world'):
    print('Hello, %s.' % name)
_Hello = type('Hello', (object,), dict(hello=fn))

In [70]:
h = _Hello()
h.hello()

Hello, world.


In [71]:
print(type(Hello))
print(type(h))

<class 'type'>
<class '__main__.Hello'>
